In [ ]:
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import requests
import boto3


# 1. DONE get one day's taxi data
# 2. DONE get one day's wheather data
# 3. DONE upload to S3 (raw data/to_processed/weather data and raw data/to_processed/taxi data)
# 4. create functions - organize the code
# 5. Creating a trigger


def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=2)
    formatted_datetime = current_datetime.strftime("%Y-%m-%d")
    
    taxi_url = (
        f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
        f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
        f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    )
    headers = {"X-App-Token": os.environ.get("CHICAGO_KEY")}
    
    response = requests.get(taxi_url)
    taxi_data = response.json()

    
    weather_url = "https://archive-api.open-meteo.com/v1/era5"
    
    params = {
        "latitude": 41.85,
        "longitude": -87.65,
        "start_date": formatted_datetime,
        "end_date": formatted_datetime,
        "hourly": "temperature_2m,wind_speed_10m,precipitation,rain"
    }
    
    responser = requests.get(weather_url, params=params)
    weather_data = response.json()
    
    #print(taxi_data)
    #print(weather_data)
    
    client = boto3.client("s3")
    client.put_object(
        Bucket="cubix-chicago-taxi-goga",
        Key=f"raw_data/to_processed/weather_data/test_upload.json",
        Body=json.dumps(weather_data)
        )
    
    client.put_object(
        Bucket="cubix-chicago-taxi-goga",
        Key=f"raw_data/to_processed/taxi_data/test_upload.json",
        Body=json.dumps(taxi_data)
        )